In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from itertools import chain
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from matplotlib.colors import Normalize
import seaborn as sns
import os

def compute_nearest_neighbors(coordinates, num_neighbors=20):
    tree = NearestNeighbors(n_neighbors=num_neighbors, metric='euclidean')
    tree.fit(coordinates.loc[:, ['x', 'y']]) #I do it like this cause i keep the cell id info too
    #get the row index (the row names) of the dataframe
    neighbor_index = tree.kneighbors(return_distance=False)
    #cellids = coordinates.index[neighbor_index.reshape(-1)]
    cellids = coordinates.loc[coordinates.index[neighbor_index.reshape(-1)], ['cellID']].values.flatten()
    cellindexes = neighbor_index.reshape(-1)
    global res
    res = dict(zip(cellindexes, cellids))
    distances, neighbors = tree.kneighbors()
    return neighbors

def aggregate_neighbors(neighbors, labels):
    """Takes an adjacency matrix and labels and aggregates the neighbors of each entry"""
    # M = adata.obsp[f'{connectivities_key}_connectivities']\n",
    # adjacency_list = adjacency_matrix.tolil()\n",
    num_cells, num_neighbors = neighbors.shape
    df = pd.DataFrame({
        'id': list(chain(*[ [i] * num_neighbors for i in range(num_cells)])),
        'neighbor': neighbors.reshape(-1),
        'label': labels[neighbors.reshape(-1)]
    })
    return pd.crosstab(df.id, df.label)

def cluster_cells(df, num_cluster=10):
    cl = KMeans(n_clusters=num_cluster, n_init = 'auto')
    pred = cl.fit_predict(df)
    return pd.DataFrame({'cluster': pred})

data = pd.read_csv('<>', index_col = False) #File will be provided in Zenodo
neighbors = compute_nearest_neighbors(data.loc[:, ['x', 'y', 'cellID']], num_neighbors=20)
matrix = aggregate_neighbors(neighbors, data.name.values)
matrix.head()

#Run k means clustering for a range of 2-20 neighborhoods i.e. num_cluster
for x in range(2, 20):
    neighborhoods = cluster_cells(matrix, num_cluster=x)
    #save in matrix w indicating the k number
    matrix['k' + str(x)] = neighborhoods

#Create a column that the ID and neighbor match the original cell annotations
cellID = []
notFound = []
for value in matrix.index:#matrix["id"]
    if value in res.keys():
        cellID.append(res[value])
    else:
        cellID.append("NA") #not all cells have neighbors?
        notFound.append(value)
        
matrix["cellID"] = cellID

matrix.to_csv(path_or_buf = 'CODEX_nhoodAnalysis.csv', index=True) #This output is being used for Figure 1